In [1]:
#GET THE SHAPEFILE OF BLOCK GROUPS IN NEW YORK STATE FROM .SHP's DBF FILE TO A CSV
import csv
from dbfpy import dbf
import os
import sys
 
filename = '/Users/danielmsheehan/Dropbox/GIS/Data/Census/census_2010/block_group/tl_2015_36_bg/tl_2015_36_bg.dbf'

if filename.endswith('.dbf'):
    print "Converting %s to csv" % filename
    csv_fn = filename[:-4]+ ".csv"
    with open(csv_fn,'wb') as csvfile:
        in_db = dbf.Dbf(filename)
        out_csv = csv.writer(csvfile)
        names = []
        for field in in_db.header.fields:
            names.append(field.name)
        out_csv.writerow(names)
        for rec in in_db:
            out_csv.writerow(rec.fieldData)
        in_db.close()
        print "Done..."
else:
  print "Filename does not end with .dbf"
#from https://gist.github.com/bertspaan/8220892

Converting /Users/danielmsheehan/Dropbox/GIS/Data/Census/census_2010/block_group/tl_2015_36_bg/tl_2015_36_bg.dbf to csv
Done...


In [2]:
import pandas as pd 

theBlockGroupCSV = filename.replace('.dbf','.csv')

df = pd.read_csv(theBlockGroupCSV)

dfBlockGroup = df[(df.COUNTYFP == 59)] #County 059 is Nassau County

dfBlockGroup.head(10)

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
1232,36,59,303700,3,360593037003,Block Group 3,G5030,S,641365,0,40.746295,-73.627806
1233,36,59,406501,4,360594065014,Block Group 4,G5030,S,710982,0,40.721855,-73.627570
1234,36,59,407600,3,360594076003,Block Group 3,G5030,S,188905,0,40.747186,-73.559084
1235,36,59,408500,2,360594085002,Block Group 2,G5030,S,359117,0,40.702905,-73.533934
1236,36,59,409500,2,360594095002,Block Group 2,G5030,S,428980,0,40.713864,-73.488764
1792,36,59,404400,4,360594044004,Block Group 4,G5030,S,173425,0,40.721153,-73.713072
1793,36,59,404500,2,360594045002,Block Group 2,G5030,S,329924,0,40.725300,-73.696306
1794,36,59,404700,1,360594047001,Block Group 1,G5030,S,424798,0,40.730916,-73.684759
1795,36,59,404700,3,360594047003,Block Group 3,G5030,S,228482,0,40.717291,-73.685356
1796,36,59,404901,3,360594049013,Block Group 3,G5030,S,175553,0,40.703370,-73.722574


In [4]:
import json
import urllib2
from pprint import pprint

listOfGeoids = dfBlockGroup["GEOID"].tolist()

listOfGeoids = listOfGeoids[678:]

variablesWanted = ['GEOID','TRANSIT_INDEX']

df = pd.DataFrame(columns=variablesWanted)

variablesURL = '%2C'.join(variablesWanted)

urlPart1 = 'http://egis.hud.gov/ArcGIS/rest/services/oshc/Fhea/MapServer/5/query?text=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&objectIds=&where=GEOID+%3D+%27'
urlPart3 = '%27&time=&returnCountOnly=false&returnIdsOnly=false&returnGeometry=false&maxAllowableOffset=&outSR=&outFields='
urlPart5 = '&f=pjson'

for i in listOfGeoids: #maybe create larger lists of BlockGroup GEOIDS, hard b/c 1000 limit, and there's more than that of Block Groups
    #print i
    i = str(i)
    urlPart2 = i
    urlPart4 = variablesURL
    
    theURL = urlPart1 + urlPart2 + urlPart3+ urlPart4 + urlPart5
    #print theURL
    
    try:
        data = json.load(urllib2.urlopen(theURL))
        jsonGeoid = data[u'features'][0][u'attributes'][u'GEOID']
        jsonTransit_index = data[u'features'][0][u'attributes'][u'transit_index']
    
    except: #okay, somethings wrong, or the Block Group has no transit value (its water or something)
        jsonGeoid = i
        jsonTransit_index = -999 #FOR BLOCK GROUPS WITHOUT TRANSIT_INDEX OR IF TIMEOUT!!!! REVIEW YOUR OUTPUT
        
    print 'BlockGroup: ' + str(jsonGeoid) + ' - the transit index is: ' + str(jsonTransit_index)
    
    dfAppend = pd.DataFrame([[jsonGeoid, jsonTransit_index]], columns=variablesWanted) 
    df = df.append(dfAppend)

BlockGroup: 360595183002 - the transit index is: 70
BlockGroup: 360595184001 - the transit index is: 1
BlockGroup: 360595184002 - the transit index is: 1
BlockGroup: 360595220003 - the transit index is: 80
BlockGroup: 360595220004 - the transit index is: 80
BlockGroup: 360593014002 - the transit index is: 80
BlockGroup: 360594049022 - the transit index is: 80
BlockGroup: 360594050001 - the transit index is: 1
BlockGroup: 360594050002 - the transit index is: 1
BlockGroup: 360594051002 - the transit index is: 80
BlockGroup: 360594051003 - the transit index is: 80
BlockGroup: 360594051004 - the transit index is: 80
BlockGroup: 360594051006 - the transit index is: 80
BlockGroup: 360594051007 - the transit index is: 80
BlockGroup: 360594052002 - the transit index is: 1
BlockGroup: 360594093001 - the transit index is: 70
BlockGroup: 360594121004 - the transit index is: 80
BlockGroup: 360594123012 - the transit index is: 90
BlockGroup: 360594123022 - the transit index is: 90
BlockGroup: 36059

In [5]:
df.to_csv('data/geoid_hud_transit_index.csv',index=False)

print df.head(1200)

           GEOID  TRANSIT_INDEX
0   360595183002             70
0   360595184001              1
0   360595184002              1
0   360595220003             80
0   360595220004             80
0   360593014002             80
0   360594049022             80
0   360594050001              1
0   360594050002              1
0   360594051002             80
0   360594051003             80
0   360594051004             80
0   360594051006             80
0   360594051007             80
0   360594052002              1
0   360594093001             70
0   360594121004             80
0   360594123012             90
0   360594123022             90
0   360594124005            100
0   360594125001             70
0   360594125003             80
0   360594126003             80
0   360594129004             80
0   360594129002             90
0   360594135003             80
0   360594136004              1
0   360594160004             80
0   360594052004             80
0   360594053013            100
..      

Example of a URL getting data from: WHERE GEOID = 360593028004

###[Transit Index Info] (http://egis.hud.gov/ArcGIS/rest/services/oshc/Fhea/MapServer/5)

http://egis.hud.gov/ArcGIS/rest/services/oshc/Fhea/MapServer/5/query?text=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&objectIds=&where=GEOID+%3D+%27360593028004%27&time=&returnCountOnly=false&returnIdsOnly=false&returnGeometry=false&maxAllowableOffset=&outSR=&outFields=GEOID%2CTRANSIT_INDEX&f=pjson

In [58]:
listOfGeoids[678:]

[360595183001,
 360595183002,
 360595184001,
 360595184002,
 360595220003,
 360595220004,
 360593014002,
 360594049022,
 360594050001,
 360594050002,
 360594051002,
 360594051003,
 360594051004,
 360594051006,
 360594051007,
 360594052002,
 360594093001,
 360594121004,
 360594123012,
 360594123022,
 360594124005,
 360594125001,
 360594125003,
 360594126003,
 360594129004,
 360594129002,
 360594135003,
 360594136004,
 360594160004,
 360594052004,
 360594053013,
 360594053021,
 360594053023,
 360594054001,
 360594054004,
 360594054005,
 360594102001,
 360594102003,
 360594103001,
 360594103002,
 360594104001,
 360594104003,
 360594105005,
 360594105006,
 360594105004,
 360594161003,
 360594164013,
 360594164021,
 360594164022,
 360594168011,
 360594169003,
 360595170002,
 360595172003,
 360595172005,
 360595173012,
 360595173013,
 360594106002,
 360594106003,
 360594106007,
 360594147003,
 360594148001,
 360594148003,
 360594148004,
 360594148005,
 360594149002,
 360594149003,
 360594149